In [ ]:
using Pkg
Pkg.activate("..")
Pkg.instantiate()
Pkg.update()
# Pkg.upgrade_manifest()

In [ ]:
using Cropbox
using Test
# using Plots
using TimeZones
using DataFramesMeta
using CSV
# Cropbox.Interact.WebIO.setup(:ijulia)

In [ ]:
obs_farm_k = CSV.read("$(@__DIR__)/../data/Obs_FarmK.csv", DataFrame) |> unitfy

In [ ]:
include("../src/Garlic.jl")

import Dates

In [ ]:
# Cropbox.dependency(Garlic.Model)

In [ ]:
tz = tz"Asia/Seoul"

Cucumber = @config (
    :Phenology => (
        optimal_temperature = 25, # Topt
        ceiling_temperature = 30, # Tceil
        critical_photoperiod = 12, # critPPD
        # initial_leaves_at_harvest = 5, # ILN
        maximum_leaf_initiation_rate = 1.79147, # LIR
        maximum_emergence_rate = 0.2,
        maximum_phyllochron_asymptote = 1.27984, # LTARa
        leaves_generic = 50, # GLN
    ),
    :Leaf => (
        maximum_elongation_rate = 4.70, # LER
        minimum_length_of_longest_leaf = 38.0, # LL
        stay_green = 8,
        length_to_width_ratio = 0.6,
    ),
    :Carbon => (
        maintenance_respiration_coefficient = 0.012, # Rm
        synthesis_efficiency = 0.8, # Yg
    ),
    :Meta => (; cultivar = :Cucumber),
    :Plant => (initial_planting_density = 20,),
);

In [ ]:
Calibrate = @config(Cucumber, (
    :Phenology => (
        planting_date = ZonedDateTime(2021, 7, 1, tz"Asia/Seoul"), # Y1 sow
        emergence_date = ZonedDateTime(2021, 7, 15, tz"Asia/Seoul"), # Y1 emg
        first_flowering_date = ZonedDateTime(2021, 11, 5, tz"Asia/Seoul"),
    ),
    :Meta => (
        planting_group = 2,
        year = 2021,
    ),
    :Calendar => (
        init = ZonedDateTime(2021, 7, 1, tz"Asia/Seoul"),
        last = ZonedDateTime(2022, 6, 30, tz"Asia/Seoul"),
    ),
    :Weather => (
        store = Garlic.loadwea("$(@__DIR__)/../data/SwtPep-Env.wea", tz"Asia/Seoul"),
    ),
));

In [ ]:
cali = simulate(Garlic.Model;
    config=Calibrate,
    stop="calendar.count",
    snap=s -> Dates.hour(s.calendar.time') == 12,
)
# @test r.leaves_initiated[end] > 0

plot_veg = visualize(cali, :DAP, [:leaves_real], kind=:line)
visualize!(plot_veg, obs_farm_k, :DAP, :leaf_count) |> display # Fig. 3.D
plot_rep = visualize(cali, :DAP, [:flowers_appeared, :fruits_appeared], kind=:line)
visualize!(plot_rep, obs_farm_k, :DAP, [:flower_count, :fruit_count]) |> display
plot_area = visualize(cali, :DAP, :green_leaf_area, kind=:line)
visualize!(plot_area, obs_farm_k, :DAP, [:leaf_area]) |> display
plot_mass = visualize(cali, :DAP, [:leaf_mass], kind=:line)
visualize!(plot_mass, obs_farm_k, :DAP, [:leaf_drymass]) |> display

In [ ]:
f(s) = s.DAP' in obs_farm_k.DAP && Dates.hour(s.calendar.time') == 12

calibrate(Garlic.Model, obs_farm_k;
    config=Calibrate,
    stop="calendar.count",
    index=:DAP,
    target=:leaf_count => :leaves_real,
    parameters= :Phenology => (;
        LTARa_max = (0.4, 1.8),
        LIR_max = (0.4, 1.8),
    ),
    snap=f,
    optim=(:MaxSteps => 20,),
)

In [ ]:
evaluate(Garlic.Model, obs_farm_k; 
    config = Calibrate,
    index = :DAP => :DAP,
    stop="calendar.count",
    snap=s -> Dates.hour(s.calendar.time') == 12,
    target = (
        :leaf_count => :leaves_real,
    ),
    metric = :rmse) |> display

In [ ]:
# cali = Cropbox.deunitfy(cali)
# r = Cropbox.deunitfy(r)

# CSV.write("$(@__DIR__)/../data/Model_Output(Calibrate).csv", cali)
# CSV.write("$(@__DIR__)/../data/Model_Output.csv(Validate).csv", r)

In [ ]:
# manipulate(Garlic.Model, :DAP, [:leaves_real];
#     config = Cucumber,
#     parameters = :Phenology => (;
#         optimal_temperature = 10:40,
#         LTARa_max = 0.5:2.5,
#         LIR_max = 0.5:2.5,
#     ),
#     stop = "calendar.count",
#     kind = :line,
# )